In [1]:
from tensorflow.keras.models import Model   
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [2]:
import numpy as np

In [3]:
vgg= VGG16(include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 90s 2us/step


In [4]:
# vgg.summary()

In [4]:
def model_build():
    input_layer=Input(shape=(120,120,3))
    vgg=VGG16(include_top=False)(input_layer)

    classif=GlobalMaxPooling2D()(vgg)
    class1=Dense(2000, activation='relu')(classif)
    class2=Dense(1, activation='sigmoid')(class1)

    regress=GlobalMaxPooling2D()(vgg)
    reg1=Dense(2000, activation='relu')(regress)
    reg2=Dense(4, activation='sigmoid')(reg1)

    faceDetector=Model(inputs=input_layer, outputs=[class2, reg2])
    return faceDetector

In [5]:
faceDetector=model_build()

In [6]:
faceDetector.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 120, 120,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 3, 3, 512) │ 14,714,688 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 2000)      │  1,026,000 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 2000)      │  1,026,000 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      2,001 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 4)         │      8,004 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,776,693 (64.00 MB)

 Trainable params: 16,776,693 (64.00 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import tensorflow as tf

train=tf.data.Dataset.load('../saved_train')
val=tf.data.Dataset.load('../saved_val')


In [9]:
X,y=train.as_numpy_iterator().next()

In [43]:
X.shape

(10, 120, 120, 3)

In [10]:
classes, coordinates=faceDetector.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [45]:
# classes

In [11]:
batches_per_epoch=len(train)
initial_learning_rate=0.0001
decay_steps=batches_per_epoch
decay_rate=0.9

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True  # If True, decay happens at discrete intervals
)


In [12]:
opt=tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [14]:
def compute_localization_loss(yTrue, yPred):
    coordinate=tf.reduce_sum(tf.square(yTrue[:,:2]-yPred[:,:2]))
    heightTrue=yTrue[:,3]-yTrue[:,1]
    widthTrue=yTrue[:,2]-yTrue[:,0]

    heightPred=yPred[:,3]-yPred[:,1]
    widthPred=yPred[:,2]-yPred[:,0]

    size=tf.reduce_sum(tf.square(widthTrue-widthPred) + tf.square(heightTrue-heightPred))

    return coordinate+size

In [15]:
classLoss=tf.keras.losses.BinaryCrossentropy()
regressLoss=compute_localization_loss

In [40]:
# compute_localization_loss(y[1], coordinates)
# classLoss(y[0],classes)

In [102]:
print("Shape of X:", X.shape)  # Should print (batch_size, 120, 120, 3)
print("Shape of y[0] (class labels):", y[0].shape)  # Should print (batch_size, num_classes)
print("Shape of y[1] (bounding boxes):", y[1].shape)  # Should print (batch_size, num_boxes, 4)

Shape of X: (10, 120, 120, 3)
Shape of y[0] (class labels): (10, 1)
Shape of y[1] (bounding boxes): (10, 4)


In [117]:
class FaceDetector(Model):
    def __init__(self, faceDetector, **kwargs):
        super().__init__(**kwargs)
        self.model=faceDetector

    def compile(self, opt, classLoss, regressLoss, **kwargs):
            # super().compile(**kwargs)
            # self.opt=opt
            # self.cLoss=classLoss
            # self.lLoss=regressLoss

            super().compile(
                  optimizer=opt,
                  loss=[classLoss,regressLoss],
                  **kwargs
            )
            self.opt=opt
            self.cLoss=classLoss
            self.lLoss=regressLoss

            # self.cLoss=loss.get('classification')
            # self.lLoss=loss.get('regression')
            # super().compile(optimizer=opt, loss={'classification': self.cLoss, 'regression': self.lLoss},**kwargs)
            # self.opt=opt

    def train_step(self, batch, **kwargs):
        X,y=batch
        # Ensure the shape of X is (batch_size, 120, 120, 3)
        X = tf.ensure_shape(X, [None, 120, 120, 3])

        y = list(y)

        # # Ensure y has the expected shape (batch_size, num_classes) for class labels
        y[0] = tf.ensure_shape(y[0], [None, 1])  # Adjust num_classes as needed
        y[1] = tf.ensure_shape(y[1], [None, 4])  # Adjust num_boxes as needed

        y = tuple(y)


        # print(f"Rank of X: {tf.rank(X)}")
        # print(f"Rank of y: {tf.rank(y)}")
        with tf.GradientTape()  as tape:
              classes,coordinates= self.model(X,training=True)
              classLoss=self.cLoss(y[0],classes)
              localizationLoss=self.lLoss(tf.cast(y[1],tf.float32),coordinates)
              totalLoss=0.5*classLoss+localizationLoss
              grad=tape.gradient(totalLoss,self.model.trainable_variables)

        opt.apply_gradients(zip(grad,self.model.trainable_variables))   
        return {'totalLoss':totalLoss, 'classLoss':classLoss, 'regressLoss':localizationLoss}

    def test_step(self, batch, **knwargs):
              
            X,y=batch

            # Ensure the shape of X is (batch_size, 120, 120, 3)
            X = tf.ensure_shape(X, [None, 120, 120, 3])

            y = list(y)

            # # Ensure y has the expected shape (batch_size, num_classes) for class labels
            y[0] = tf.ensure_shape(y[0], [None, 1])  # Adjust num_classes as needed
            y[1] = tf.ensure_shape(y[1], [None, 4])  # Adjust num_boxes as needed

            y = tuple(y)

            classes,coordinates= self.model(X,training=True)
            classLoss=self.cLoss(y[0],classes)
            localizationLoss=self.lLoss(tf.cast(y[1],tf.float32),coordinates)
            totalLoss=0.5*classLoss+localizationLoss

            return {'totalLoss':totalLoss, 'classLoss':classLoss, 'regressLoss':localizationLoss}
    
    def call(self, X, **kwargs):
          return self.model(X, **kwargs)
            



In [118]:
model=FaceDetector(faceDetector)

In [119]:
# print(model)

In [120]:
model.compile(opt, classLoss, regressLoss)
# model.compile(opt, loss={'classification': classLoss, 'regression': regressLoss})

In [121]:
logDir='../visulaizationData'

In [122]:
tensorBoard=tf.keras.callbacks.TensorBoard(log_dir=logDir)

In [123]:
hist=model.fit(train, epochs=30, validation_data=val, callbacks=[tensorBoard])

Epoch 1/30


64/64 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - classLoss: 0.0499 - regressLoss: 0.2596 - totalLoss: 0.2845 - val_classLoss: 0.0031 - val_regressLoss: 0.0963 - val_totalLoss: 0.0979
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - classLoss: 0.0286 - regressLoss: 0.1783 - totalLoss: 0.1926 - val_classLoss: 0.0010 - val_regressLoss: 0.0646 - val_totalLoss: 0.0651
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - classLoss: 0.0267 - regressLoss: 0.1379 - totalLoss: 0.1513 - val_classLoss: 0.0031 - val_regressLoss: 0.0285 - val_totalLoss: 0.0300
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - classLoss: 0.0292 - regressLoss: 0.1174 - totalLoss: 0.1321 - val_classLoss: 0.0024 - val_regressLoss: 0.0529 - val_totalLoss: 0.0541
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - classLoss: 0.0130 - regressLoss: 0.0753 - totalLoss: 0.0818 - val_classLoss: 3.4726e-04 - val_regressLoss: 0.0221 - val_totalLoss: 0.0223
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - classLoss: 0.0031 -

In [57]:
type(train)

tensorflow.python.data.ops.load_op._LoadDataset

In [59]:
for batch in train.take(1):
    print(batch)


(<tf.Tensor: shape=(10, 120, 120, 3), dtype=float32, numpy=
array([[[[0.3807598 , 0.38811275, 0.33063725],
         [0.40765932, 0.37922794, 0.34834558],
         [0.4266544 , 0.37224266, 0.33694854],
         ...,
         [0.47205883, 0.4127451 , 0.33970588],
         [0.47879902, 0.4317402 , 0.37193626],
         [0.45637256, 0.43137255, 0.3495098 ]],

        [[0.41832107, 0.42273283, 0.4041054 ],
         [0.41556373, 0.3959559 , 0.3788603 ],
         [0.43229166, 0.39442402, 0.3552696 ],
         ...,
         [0.46415442, 0.41317403, 0.3504289 ],
         [0.46801472, 0.41911766, 0.37199754],
         [0.46519607, 0.41715688, 0.36531863]],

        [[0.425     , 0.4708946 , 0.46746323],
         [0.41458333, 0.43131128, 0.4033701 ],
         [0.42990196, 0.4028799 , 0.36452207],
         ...,
         [0.4543505 , 0.4033701 , 0.3362745 ],
         [0.47591913, 0.41060048, 0.33970588],
         [0.48112744, 0.4146446 , 0.34503677]],

        ...,

        [[0.46372548, 0.5892157 

In [125]:
model.save("../Model/model.keras")

c:\Users\rajus\OneDrive\Desktop\Rahul Project\AIML\myenv\Lib\site-packages\keras\src\saving\saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)
